# `pandas` and csv files

In [ ]:
# read data into a dataframe
df_T = pd.read_csv('../data/allT_updated.csv', index_col=0)
df_T.head()

In [ ]:
df_T['Temperature'].plot()

In [ ]:
df_T[df_T['YEAR']==2014]['Temperature'].plot()

# `xarray` and Satellite data

## Import your modules or packages

In [ ]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Get some data and check what's in it

In [ ]:
# assign an url of where the data is in NETCDF format
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/2011/305/20111101120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'

# open the url with xarray!
ds_sst = xr.open_dataset(url)

# an object or variable in the last line display what is in it
ds_sst 

In [ ]:
# Display values (or data) from a variable in the Dataset
ds_sst.analysed_sst.data

In [ ]:
# Display attributes of a variable in the Dataset
ds_sst.analysed_sst.attrs

In [ ]:
# Easy plot of a variable in a Dataset
ds_sst.analysed_sst.plot()

In [ ]:
# and now the mask
ds_sst.mask.plot()

In [ ]:
# selecting the ocean mask
mask_ocean = ds_sst.mask.where(ds_sst.mask==1)
mask_ocean.plot()

In [ ]:
# get sea surface temperature with mask, in Celsius
sst_global = ds_sst.analysed_sst*mask_ocean
sst_global -= 273.15
sst_global.plot()
sst_global

In [ ]:
# Lets focus in our neigborhood
sst_california = sst_global.sel(lat=slice(30,45),lon=slice(-130,-115))
sst_california.plot()

In [ ]:
# take the average of a box
sst_cal_pt = sst_global.sel(lat=slice(38,42),lon=slice(-126,-123)).mean()
sst_cal_pt.data

In [ ]:
# an example to extract a time series from a netcdf - i can use a hadley file
#url2 = 'https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST1_SST_update.nc.gz'
ds_had = xr.open_dataset(url2)


## [MUR SST](https://podaac.jpl.nasa.gov/Multi-scale_Ultra-high_Resolution_MUR-SST) [AWS Public dataset program](https://registry.opendata.aws/mur/) 

### Access the MUR SST Zarr store which is in an s3 bucket.  

![image](https://podaac.jpl.nasa.gov/Podaac/thumbnails/MUR-JPL-L4-GLOB-v4.1.jpg)

We will start with my favorite Analysis Ready Data (ARD) format: [Zarr](https://zarr.readthedocs.io/en/stable/).  Using data stored in Zarr is fast, simple, and contains all the metadata normally in a netcdf file, so you can figure out easily what is in the datastore.  

- Fast - Zarr is fast because all the metadata is consolidated into a .json file.  Reading in massive datasets is lightning fast because it only reads the metadata and does read in data until it needs it for compute.ac

- Simple - Filenames?  Who needs them? Who cares?  Not I.  Simply point your read routine to the data directory.

- Metadata - all you want!

In [ ]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

In [ ]:
%%time
file_location = 's3://mur-sst/zarr'

ikey = fsspec.get_mapper(file_location, anon=True)

ds_sst = xr.open_zarr(ikey,consolidated=True)

ds_sst

### Read entire 10 years of data at 1 point.

Select the ``analysed_sst`` variable over a specific time period, `lat`, and `lon` and load the data into memory.  This is small enough to load into memory which will make calculating climatologies easier in the next step.

In [ ]:
%%time

sst_timeseries = ds_sst['analysed_sst'].sel(time = slice('2010-01-01','2020-01-01'),
                                            lat  = 47,
                                            lon  = -145
                                           ).load()

sst_timeseries.plot()

### The anomaly is more interesting...  

Use [.groupby](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html#xarray-dataarray-groupby) method to calculate the climatology and [.resample](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html#xarray-dataset-resample) method to then average it into 1-month bins.
- [DataArray.mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray-dataarray-mean) arguments are important! Xarray uses metadata to plot, so keep_attrs is a nice feature.  Also, for SST there are regions with changing sea ice.  Setting skipna = False removes these regions. 

In [ ]:
%%time

sst_climatology = sst_timeseries.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)

sst_anomaly = sst_timeseries.groupby('time.dayofyear')-sst_climatology

sst_anomaly_monthly = sst_anomaly.resample(time='1MS').mean(keep_attrs=True,skipna=False)

#plot the data
sst_anomaly.plot()
sst_anomaly_monthly.plot()
plt.axhline(linewidth=2,color='k')
plt.savefig('./figures/sst_anomaly.png')